# Optuna

In [1]:
!pip install optuna

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.7 kB 2.7 MB/s eta 0:00:01
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/364.4 kB ? eta -:--:--
   --------------------------------------  358.4/364.4 kB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 364.4/364.4 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/233.5 kB ? eta -:--:--
   --------------------------------------- 233.5/233.5 kB 14.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----------------------------- ---------- 1.5/2.1 MB 32.5 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 26.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 4.3 MB/s eta 0:00:00
   ------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\CALEO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install optuna-dashboard

   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.4 MB 445.2 kB/s eta 0:00:19
   ---------------------------------------- 0.1/8.4 MB 845.5 kB/s eta 0:00:10
   - -------------------------------------- 0.2/8.4 MB 1.5 MB/s eta 0:00:06
   -- ------------------------------------- 0.5/8.4 MB 2.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.1/8.4 MB 4.9 MB/s eta 0:00:02
   ----------- ---------------------------- 2.4/8.4 MB 8.4 MB/s eta 0:00:01
   -------------- ------------------------- 3.1/8.4 MB 10.6 MB/s eta 0:00:01
   -------------- ------------------------- 3.1/8.4 MB 10.6 MB/s eta 0:00:01
   -------------------- ------------------- 4.2/8.4 MB 10.4 MB/s eta 0:00:01
   ----------------------------- ---------- 6.1/8.4 MB 13.4 MB/s eta 0:00:01
   ---------------------------------- ----- 7.3/8.4 MB 15.6 MB/s eta 0:00:01
   --------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\CALEO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import os
import numpy as np
import pandas as pd

def load_txt_files_from_folders(root_directory):
    """
    Percorre todas as pastas dentro do diretório raiz e carrega arquivos .txt de cada pasta.

    Args:
        root_directory (str): Caminho para o diretório raiz.

    Returns:
        dict: Um dicionário com as informações carregadas.
              Chave = 'nome_da_pasta/nome_do_arquivo'
              Valor = (cabeçalho, np.array com os dados)
    """
    relevant_features_data = {}

    # Percorre todas as subpastas dentro do diretório raiz
    for folder_name in os.listdir(root_directory):
        folder_path = os.path.join(root_directory, folder_name)

        # Verifica se é uma pasta
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                if filename.endswith('.txt'):
                    file_path = os.path.join(folder_path, filename)
                    try:
                        # Lendo os dados do arquivo
                        with open(file_path, 'r') as file:
                            lines = file.readlines()
                            header = lines[0].strip()  # Mantém o cabeçalho
                            data = [line.strip().split(',') for line in lines[1:] if line.strip()]  # Supondo CSV-like

                            # Adiciona ao dicionário com chave como 'pasta/arquivo'
                            key = f"{folder_name}/{filename}"
                            relevant_features_data[key] = (header, np.array(data, dtype=float))

                    except Exception as e:
                        print(f"Erro ao processar o arquivo {file_path}: {e}")
    
    return relevant_features_data


# Exemplo de uso:
root_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output3"
datasets = load_txt_files_from_folders(root_directory)

# Exibindo o resumo dos arquivos carregados
for key, (header, data) in datasets.items():
    print(f"Arquivo processado: {key}")
    print(f"Header: {header}")
    print(f"Shape dos dados: {data.shape}")
    print("-" * 50)


Arquivo processado: 1-saida/cocomo81.txt
Header: rely,data,modp,loc,actual
Shape dos dados: (63, 5)
--------------------------------------------------
Arquivo processado: 1-saida/encoded_china.txt
Header: AFP,Input,File,Added,N_effort,Effort
Shape dos dados: (499, 6)
--------------------------------------------------
Arquivo processado: 1-saida/encoded_desharnais.txt
Header: Transactions,Entities,PointsAdjust,Envergure,Language_3,Effort
Shape dos dados: (81, 6)
--------------------------------------------------
Arquivo processado: 1-saida/encoded_maxwell.txt
Header: size_D,duration_D,nlan,t07,t10,t11,t14,t15,app_InfServ,effort_D
Shape dos dados: (63, 10)
--------------------------------------------------
Arquivo processado: 10-saida/cocomo81.txt
Header: loc,Generated_Var_1,Generated_Var_2,Generated_Var_3,Generated_Var_4,actual
Shape dos dados: (63, 6)
--------------------------------------------------
Arquivo processado: 10-saida/encoded_china.txt
Header: N_effort,Generated_Var_1,Gener

In [2]:
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score

# Função para encontrar os melhores hiperparâmetros usando Optuna
def objective(trial, X, y):
    """
    Função objetivo para o Optuna.

    Args:
        trial (optuna.trial.Trial): Objeto de tentativa do Optuna.
        X (np.ndarray): Variáveis independentes.
        y (np.ndarray): Variável dependente.

    Returns:
        float: Métrica a ser minimizada ou maximizada.
    """
    # Hiperparâmetros a serem otimizados
    fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
    positive = trial.suggest_categorical("positive", [True, False])

    # Configurar validação cruzada
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    r2_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Criar o modelo
        model = LinearRegression(fit_intercept=fit_intercept, positive=positive)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calcular R²
        r2_scores.append(r2_score(y_test, y_pred))

    return -np.mean(r2_scores)  # Negativo porque Optuna minimiza a função objetivo

# Criar um estudo Optuna para cada dataset
best_params_summary = []

for key, (header, data) in datasets.items():
    print(f"Otimizando hiperparâmetros para o dataset: {key}")

    X = data[:, :-1]
    y = data[:, -1]

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=50)

    # Salvar os melhores hiperparâmetros e o melhor valor objetivo
    best_params_summary.append({
        "Dataset": key,
        "Best Params": study.best_params,
        "Best R²": -study.best_value  # Tornar positivo novamente
    })

    print(f"Melhores hiperparâmetros para {key}: {study.best_params}")
    print(f"Melhor R² para {key}: {-study.best_value}")
    print("-" * 50)

# Converter o resumo para DataFrame e salvar
results_df = pd.DataFrame(best_params_summary)
results_df.to_excel("optuna_hyperparameter_tuning_results.xlsx", index=False)
print("Resultados salvos em optuna_hyperparameter_tuning_results.xlsx")


[I 2024-12-19 07:22:41,791] A new study created in memory with name: no-name-fdafa410-5279-497f-a747-a99cb67583ec
[I 2024-12-19 07:22:41,812] Trial 0 finished with value: 1.0978008294111463 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:41,820] Trial 1 finished with value: 1.2313568505187893 and parameters: {'fit_intercept': False, 'positive': False}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:41,829] Trial 2 finished with value: 2.4875991132493422 and parameters: {'fit_intercept': True, 'positive': True}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:41,833] Trial 3 finished with value: 1.2313568505187893 and parameters: {'fit_intercept': False, 'positive': False}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:41,837] Trial 4 finished with value: 1.2313568505187893 and parameters: {'fit_intercept': False, 'positive': False}. Best is tria

Otimizando hiperparâmetros para o dataset: 1-saida/cocomo81.txt


[I 2024-12-19 07:22:41,983] Trial 30 finished with value: 1.0978008294111463 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:41,989] Trial 31 finished with value: 1.0978008294111463 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:41,995] Trial 32 finished with value: 1.0978008294111463 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:42,002] Trial 33 finished with value: 1.0978008294111463 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:42,004] Trial 34 finished with value: 1.0978008294111463 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 0 with value: 1.0978008294111463.
[I 2024-12-19 07:22:42,013] Trial 35 finished with value: 2.487599113247104

Melhores hiperparâmetros para 1-saida/cocomo81.txt: {'fit_intercept': False, 'positive': True}
Melhor R² para 1-saida/cocomo81.txt: -1.0978008294111463
--------------------------------------------------
Otimizando hiperparâmetros para o dataset: 1-saida/encoded_china.txt


[I 2024-12-19 07:22:42,290] Trial 28 finished with value: -0.9743264932186891 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 5 with value: -0.9743264932186891.
[I 2024-12-19 07:22:42,296] Trial 29 finished with value: -0.9743264932186891 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 5 with value: -0.9743264932186891.
[I 2024-12-19 07:22:42,302] Trial 30 finished with value: -0.9743264932186891 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 5 with value: -0.9743264932186891.
[I 2024-12-19 07:22:42,308] Trial 31 finished with value: -0.9743264932186891 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 5 with value: -0.9743264932186891.
[I 2024-12-19 07:22:42,314] Trial 32 finished with value: -0.9743264932186891 and parameters: {'fit_intercept': False, 'positive': True}. Best is trial 5 with value: -0.9743264932186891.
[I 2024-12-19 07:22:42,321] Trial 33 finished with value: -0.9743

Melhores hiperparâmetros para 1-saida/encoded_china.txt: {'fit_intercept': False, 'positive': True}
Melhor R² para 1-saida/encoded_china.txt: 0.9743264932186891
--------------------------------------------------
Otimizando hiperparâmetros para o dataset: 1-saida/encoded_desharnais.txt


RuntimeError: Maximum number of iterations reached.